# Read data set

# Imports

In [ ]:
import os
import glob
from skimage.feature import hog
import numpy as np
import cv2
from random import randint, sample
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import deque
import time

# Data import

In [ ]:
"""Raed the directory of images and store the file names"""
def read_dir(dirname, extension):
    imgtypes = os.listdir(dirname)
    results = []
    for imtype in imgtypes:
        npath= dirname + imtype + '/*.' + extension
        results.extend(glob.glob(npath))
    return results

"""Read the data set and return the image file names"""
def read_dataset():
    v_images = read_dir('vehicles/', 'png')
    nv_images = read_dir('non-vehicles/', 'png')
    return (v_images, nv_images)

# Feature Extraction/Image Processing methods

In [2]:
"""Get HOG features of the image and optionally the HOG visualization"""
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis, feature_vec):
    
    transform_sqrt=False # this does not work for -ve values in img
    block_norm='L2-Hys'
    
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm=block_norm,
                                  transform_sqrt=transform_sqrt, visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                      cells_per_block=(cell_per_block, cell_per_block), block_norm=block_norm,
                      transform_sqrt=transform_sqrt, visualise=vis, feature_vector=feature_vec)
        
        if (np.isnan(np.min(features))):
            from IPython.core.debugger import Tracer; Tracer()() 
        return features
    
"""Get the binned spatial features of the image"""  
def bin_spatial(img, size):
    # Use cv2.resize().ravel() to create the feature vector
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
    # Return the feature vector
    return features

"""Get the color histogram features of the image"""  
def color_hist(img, nbins):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

"""Convert the image from one color space to the other"""
def convert_color(image, color_space):
    if color_space == 'RGB':
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif color_space == 'HSV':
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif color_space == 'LUV':
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif color_space == 'HLS':
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    elif color_space == 'YUV':
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    elif color_space == 'YCrCb':
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    else: 
        feature_image = np.copy(image)   
    return feature_image
    
"""Extract spatial, color and/or HOG features of the images per the specifications"""
def extract_features(imgs, color_space, spatial_size,
                    hist_bins, orient, pix_per_cell, cell_per_block, hog_channel,
                    spatial_feat, hist_feat, hog_feat):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = cv2.imread(file)
        image = image.astype(np.float32)
        feature_image = convert_color(image, color_space)  

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            color_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(color_features)
        if hog_feat == True:
            # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
            
        features.append(np.concatenate(file_features))
            
    # Return list of feature vectors
    return features

"""Extract spatial,color and/or HOG features for a single image
Optionally return the HOG visualization of the image"""
def single_img_features(img, color_space, spatial_size,
                        hist_bins, orient, 
                        pix_per_cell, cell_per_block, hog_channel, vis,
                        spatial_feat, hist_feat, hog_feat):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    feature_image = convert_color(img, color_space) 
        
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features, hog_image = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=vis, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    if vis == True:
        #9) Return concatenated array of features
        return np.concatenate(img_features), hog_image
    else:
        #9) Return concatenated array of features
        return np.concatenate(img_features)

    
"""Define a function that takes an image, 
start and stop positions in both x and y, 
window size (x and y dimensions),  
and overlap fraction (for both x and y)"""
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


"""Define a function to draw bounding boxes"""
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


"""Define a function you will pass an image 
and the list of windows to be searched (output of slide_windows())"""
def search_windows(img, windows, clf, scaler, color_space, 
                    spatial_size, hist_bins, 
                    orient, pix_per_cell, cell_per_block, hog_channel, 
                    spatial_feat, hist_feat, hog_feat):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, vis=False,
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = np.array(features).reshape(1, -1)
        test_features = scaler.transform(test_features)
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)

    #8) Return windows for positive detections
    return on_windows


"""Define a function that visualizes the given list of images and titles 
on the figure provided

Keyword arguments:
fig -- Figure to visualize on
rows - Number of image rows
cols - Number of image columns
imgs - Images to visualize
titles - Titles of the images to visualize
"""
def visualize(fig, rows, cols, imgs, titles):
    for i, img in enumerate(imgs):
        plt.subplot(rows, cols, i+1)
        plt.title(i+1)
        img_dims = len(img.shape)
        if img_dims < 3:
            plt.imshow(img, cmap='hot')
        else:
            plt.imshow(img)
        plt.title(titles[i])

# Feature parameters

In [3]:
# feature parameters
color_space = 'YCrCb'
orient = 9
pix_per_cell=8
cell_per_block=2
hog_channel='ALL'
spatial_size = (32,32)
hist_bins=32
spatial_feat=True
hist_feat=True
hog_feat=True

In [ ]:
%matplotlib inline

"""Visualize some samples of the data set by plotting them 
and their corresponding HOG image

Keyword argumets:
car_images -- Car images
not_car_images - Images that are not of cars
num_samples - Number of samples to visualize
"""
def sample_feature_visualization(car_images, not_car_images, num_samples):
    for i in range(0, num_samples):
        car_image = cv2.imread(car_images[randint(0, len(car_images))])
        rgb_car = cv2.cvtColor(car_image, cv2.COLOR_BGR2RGB)
        car_image = np.array(car_image).astype(np.float32)
        not_car_image = cv2.imread(not_car_images[randint(0, len(not_car_images))])
        rgb_not_car = cv2.cvtColor(not_car_image, cv2.COLOR_BGR2RGB)
        not_car_image = np.array(not_car_image).astype(np.float32)
        
        print ("Pixel Range: Min - {}. Max - {}".format(np.min(car_image), np.max(car_image)))


        car_features, car_hog_image = single_img_features(car_image, 
                                         color_space=color_space,
                                         spatial_size=spatial_size, 
                                         hist_bins=hist_bins,
                                         orient=orient,
                                         pix_per_cell=8,
                                         cell_per_block=2,
                                         hog_channel=0,
                                         vis=True,
                                         spatial_feat=spatial_feat,
                                         hist_feat=hist_feat,
                                         hog_feat=hog_feat)
        not_car_features, not_car_hog_image = single_img_features(not_car_image, 
                                         color_space=color_space,
                                         spatial_size=spatial_size, 
                                         hist_bins=hist_bins,
                                         orient=orient,
                                         pix_per_cell=8,
                                         cell_per_block=2,
                                         hog_channel=0,
                                         vis=True,
                                         spatial_feat=spatial_feat,
                                         hist_feat=hist_feat,
                                         hog_feat=hog_feat)

        images = [rgb_car, car_hog_image, rgb_not_car, not_car_hog_image]
        titles = ['Car', 'Car HOG', 'Not Car', 'Not Car HOG']
        fig = plt.figure(figsize=(12,3))
        visualize(fig, 1, 4, images, titles)

In [ ]:
(v_images, nv_images) = read_dataset()
print ("Number of vehicle images: {}".format(len(v_images)))
print ("Number of non-vehicle images: {}".format(len(nv_images)))

sample_feature_visualization(v_images, nv_images, 2)


# Training

In [ ]:
"""Train an SVC classification model on the given set of 
car and not-car images

Keyword arguments:
car_images -- Images of cars
not_car_images -- Images that are not of cars
"""
def train(car_images, not_car_images):

    test_cars = np.array(car_images)
    test_notcars = np.array(not_car_images)

    # get features
    car_features = extract_features(test_cars, 
                                 color_space=color_space,
                                 spatial_size=spatial_size, 
                                 hist_bins=hist_bins,
                                 orient=orient,
                                 pix_per_cell=pix_per_cell,
                                 cell_per_block=cell_per_block,
                                 hog_channel=hog_channel,
                                 spatial_feat=spatial_feat,
                                 hist_feat=hist_feat,
                                 hog_feat=hog_feat)
    not_car_features = extract_features(test_notcars, 
                                     color_space=color_space,
                                     spatial_size=spatial_size, 
                                     hist_bins=hist_bins,
                                     orient=orient,
                                     pix_per_cell=pix_per_cell,
                                     cell_per_block=cell_per_block,
                                     hog_channel=hog_channel,
                                     spatial_feat=spatial_feat,
                                     hist_feat=hist_feat,
                                     hog_feat=hog_feat)
    
    X = np.vstack((car_features, not_car_features)).astype(np.float64)       
    
    print ("Training feature shape: {}".format(X.shape))
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(not_car_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.1, random_state=rand_state)

    clf = SVC(C=10, kernel='rbf')
    # Check the training time for the SVC
    t=time.time()
    clf.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')

    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))
    # Check the prediction time for a single sample
    t=time.time()
    
    return (clf, X_scaler) # return the best classifier

In [ ]:
clf, scaler = train(v_images, nv_images)

# persist the model
scaler_file = 'scaler.pk'
model_file = 'model.pk'
joblib.dump(clf, model_file)
joblib.dump(scaler, scaler_file)

# Test Classifier on test images - Slow

In [4]:
load_model_file = 'model.pk'
load_scaler_file='scaler.pk'

clf = joblib.load(load_model_file)
scaler = joblib.load(load_scaler_file)

In [ ]:
# load test images
test_img_path = 'test_images/'
test_images = glob.glob(test_img_path + '*.jpg')
print ("Number of test images: {}".format(len(test_images)))

In [ ]:
%matplotlib inline

images=[]
titles = []
y_start_stop=[400,680]
overlap=0.5
xy_window=(96,96)

for img_file in test_images:
    t1 = time.time()
    img = cv2.imread(img_file)
    draw_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    
    windows = slide_window(img, x_start_stop=[None,None], y_start_stop=y_start_stop,
                          xy_window=xy_window, xy_overlap=(overlap, overlap))
    
    hot_windows = search_windows(img, windows, clf, scaler, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins,
                                orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat)
    
    window_img = draw_boxes(draw_img, hot_windows, color=(0, 0, 255), thick=6)
    images.append(window_img)
    titles.append('')
    print ("{} seconds to process one image. Windows: {}".format(time.time()-t1, len(windows)))

fig = plt.figure(figsize=(12,18))
visualize(fig, 5, 2, images, titles)

# Image processing - Sliding Window

In [5]:
"""A class that processes the given image using the sliding window
approach and draws bounding boxes around cars
"""
class ImageProcessor:
    """Constructor
    
    Keyword arguments:
    smooth_factor -- Number of heat maps to smooth output over. 
    Moving average is used
    clf -- The classifier to use
    scaler -- THe feature scaler to use
    parameters -- Parameters for the sliding window. The format is:
    an array of tuples: (scale, ystart, yend)
    threshold -- Number of detections to be considered as a car"""
    def __init__(self, smooth_factor, clf, scaler, parameters, threshold):
        self.recent_heatmaps = deque(maxlen=smooth_factor)
        self.clf = clf
        self.scaler = scaler
        self.parameters = parameters
        self.threshold = threshold
    
    """Apply a threshold on the given heat map"""
    def apply_heat_threshold(self, heatmap):
        heatmap[heatmap < self.threshold] = 0
        return heatmap
    
    """Draw bounding boxes given the labels of the image"""
    def draw_labeled_bboxes(self, img, labels):
        for car_number in range(1, labels[1] + 1):
            # find pixels for the car_number
            nonzero = (labels[0] == car_number).nonzero()

            # identify x and y
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])

            # draw a bounding box based on mix/max values
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))

            # draw the rectangle
            cv2.rectangle(img, bbox[0], bbox[1], (0, 0, 255), 6)
        
        return img

    """Find the cars in the given image and draw bounding boxes around them"""
    def find_cars(self, img):
        
        # output image in RGB
        draw_img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # input image as float
        img = img.astype(np.float32)
            
        # find the aggregate heat map for all y and scale parameters
        overall_heatmap = np.zeros_like(img[:,:,0])
        for scale,ystart,yend in self.parameters:
            heatmap = self.find_cars_single_param(img, ystart,yend,scale)
            overall_heatmap = np.add(overall_heatmap, heatmap)
                   
        # record it
        self.recent_heatmaps.append(overall_heatmap)
        # smooth out
        overall_heatmap = np.average(self.recent_heatmaps, axis=0)
        # threshold and return
        overall_heatmap = self.apply_heat_threshold(overall_heatmap)
        
        # label and bbox it
        labels = label(overall_heatmap)
        draw_img = self.draw_labeled_bboxes(draw_img, labels)
        
        return draw_img, overall_heatmap
        
    """Find the cars in the given image and return the heatmap
    
    Keyword arguments:
    img -- Image to find cars in
    ystart -- Y co-ordinate to start search from
    yend -- Y co-ordinate to end search at
    scale -- Scale of the sliding window. A scale of 1 is 64 X 64"""
    def find_cars_single_param(self, img, ystart, yend, scale):
        t = time.time()

        search_img=img[ystart:yend,:,:]
        ctrans_search=convert_color(search_img,color_space)

        if scale != 1:
            imshape = ctrans_search.shape
            # resize the image instead of resizing the boxes. The effect is the same
            ctrans_search = cv2.resize(ctrans_search, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

        # get the hog features of the scaled image
        ch1 = ctrans_search[:,:,0]
        ch2 = ctrans_search[:,:,1]
        ch3 = ctrans_search[:,:,2]
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)

        # Define blocks and steps
        width = ch1.shape[1]
        height = ch1.shape[0]
        nxcells = width // pix_per_cell
        nycells = height // pix_per_cell

        # original window size
        window = 64
        window_cells = window // pix_per_cell

        # blocks are like 1 d conv moving 1 cell at a time
        nxblocks = nxcells - cell_per_block + 1 
        nyblocks = nycells - cell_per_block + 1
        nblocks_per_window = window_cells - cell_per_block + 1 

        cells_per_step=2 # 75% overlap
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step

        #print ("nxblocks: {}, nyblocks: {}".format(nxblocks, nyblocks))
        #print ("window blocks: {}".format(nblocks_per_window))
        #print ("nxsteps: {}, nysteps: {}".format(nxsteps, nysteps))

        heatmap = np.zeros_like(img[:,:,0])
        count=0 # number of boxes
        for xb in range(nxsteps):
            for yb in range(nysteps):
                count += 1

                ypos = yb * cells_per_step
                xpos = xb * cells_per_step

                # extract HOG for this patch
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

                xleft = xpos * pix_per_cell
                ytop = ypos * pix_per_cell

                # Extract the image patch
                subimg = cv2.resize(ctrans_search[ytop:ytop+window, xleft:xleft+window], (window,window))

                # Get color features
                spatial_features = bin_spatial(subimg, size=spatial_size)
                hist_features = color_hist(subimg, nbins=hist_bins)
                # careful of the order!
                features = np.hstack((spatial_features, hist_features, hog_features)).reshape(1,-1)
                # scale features and make the prediction
                scaled_features = self.scaler.transform(features)
                prediction = self.clf.predict(scaled_features)

                if prediction == 1:
                    win_draw = np.int(window*scale)
                    xbox_left = np.int(xleft*scale)
                    xbox_right = xbox_left + win_draw
                    ytop_draw = ystart + np.int(ytop*scale)
                    ybot_draw = ytop_draw + win_draw

                    #cv2.rectangle(draw_img, 
                    #              (xbox_left, ytop_draw), 
                    #              (xbox_right, ybot_draw),
                    #              (0,0,255), 6)
                    heatmap[ytop_draw:ybot_draw, xbox_left:xbox_right] += 1

        #print("{} seconds to run for {} windows".format(time.time() - t, count))
        
        return heatmap

In [ ]:
%matplotlib inline

out_images= []
out_titles = []
params = [(1.0, 400, 480), (1.5, 400, 680), (2.5, 480, 680)]
proc = ImageProcessor(smooth_factor=1, clf=clf, scaler=scaler, parameters=params, threshold=1)
for img_file in test_images:
    img=cv2.imread(img_file)
    draw_img, heatmap = proc.find_cars(img)   
    
    out_images.append(draw_img)
    out_images.append(heatmap)
    out_titles.append(img_file[-9:])
    out_titles.append(img_file[-9:])
    
fig = plt.figure(figsize=(12,24))
visualize(fig, 8, 2, out_images, out_titles)

# Video Processing

In [6]:
"""A class to process video frames and draw 
bounding boxes on cars detected"""
class VideoFrameProcessor:
    """Constructor"""
    def __init__(self):
        params = [(1.0, 400, 480), (1.5, 400, 680), (2.5, 480, 680)]
        self.proc = ImageProcessor(smooth_factor=10, clf=clf, scaler=scaler, parameters=params, threshold=3)
        
    """Process the given video frame"""
    def process_video_frame(self, img):
        input_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        draw_img, heat_map = self.proc.find_cars(input_img)

        return draw_img

In [7]:
test_output = 'test.mp4'
clip = VideoFileClip("test_video.mp4")
vproc = VideoFrameProcessor()
test_clip = clip.fl_image(vproc.process_video_frame)
test_clip.write_videofile(test_output, audio=False)

[MoviePy] >>>> Building video test.mp4
[MoviePy] Writing video test.mp4


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [07:24<00:11, 11.87s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test.mp4 



In [ ]:
proj_output = 'project.mp4'
clip = VideoFileClip("project_video.mp4")
vproc = VideoFrameProcessor()
proj_clip = clip.fl_image(vproc.process_video_frame)
proj_clip.write_videofile(proj_output, audio=False)
